# Jet ColorMap Value

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
# -----------------------------------------------------------------------------/

# 创建灰度渐变的 LUT
num_values = 256
lut = np.arange(num_values, dtype=np.uint8)

# 将 LUT 变成正方形形状
lut_square = np.tile(lut, (int(num_values/4), 1))

# 创建 8 位灰度图像
lut_image = cv2.merge((lut_square, lut_square, lut_square))
jet = cv2.cvtColor(cv2.applyColorMap(lut_image, cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB)

# 显示图像
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)
ax.imshow(jet)
ax.set_xticks(np.arange(0, 255, 10))
ax.set_yticks([])

In [3]:
def plot_img_flow(img_dict:dict[str, np.ndarray],
                    keys:list[str], col:int=None, dpi=200):
    """
    """
    if col is None:
        col = len(keys); row=1
    else:
        row = int(len(keys)/col)
    assert row*col == len(keys), "For proper column setup, `len(keys)` should be divisible by column"
    
    fig_w, fig_h = col*4, row*4
    
    fig, axes = plt.subplots(row, col, figsize=(fig_w, fig_h), dpi=dpi)
    axes = np.array(axes) # 讓 len(keys) == 1 可以 flatten()
    
    num = 1
    for ax, k in zip(axes.flatten(), keys):
        
        im = img_dict[k]
        if im.dtype == np.bool8:
            im = np.uint8(im*255)
        
        ax.imshow(im, vmin=0, vmax=255, cmap="gray")
        if k != "blank":
            ax.set_title(f"({num}) {k}, {im.shape}", fontsize=10)
            num += 1
        ax.tick_params(axis='both', which='major', labelsize=10)

# Dev: CAM Thres Area

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import skimage as ski
from rich import print

pkg_dir = Path("./../../").resolve() # `dir_depth` to `repo_root`
if (pkg_dir.exists()) and (str(pkg_dir) not in sys.path):
    sys.path.insert(0, str(pkg_dir)) # add path to scan customized package

from modules.data.dataset.utils import parse_dataset_file_name
from modules.shared.config import load_config
from modules.shared.pathnavigator import PathNavigator
from modules.dl.tester.utils import get_history_dir
# -----------------------------------------------------------------------------/

# init components
path_navigator = PathNavigator()

# maunal variables
img_dict: dict[str, np.ndarray] = {}
img_dict["kernel_ones2x2"] = np.ones((2, 2), dtype=np.uint8)
img_dict["kernel_ones3x3"] = np.ones((3, 3), dtype=np.uint8)

# load `config`
config = load_config("6.run_cam_analysis.toml")
# [model_prediction]
model_time_stamp: str = config["model_prediction"]["time_stamp"]
model_state: str = config["model_prediction"]["state"]
# [cam_analysis]
cam_threshold = config["cam_analysis"]["threshold"]
# [cam_analysis.test]
testsample = config["cam_analysis"]["dev"]["testsample"]
# history_dir
history_dir = get_history_dir(path_navigator, model_time_stamp, model_state)
cam_result_root = history_dir.joinpath("cam_result")

# load `training_config`
training_config = load_config(history_dir.joinpath("training_config.toml"))
# [dataset]
dataset_seed_dir: str = training_config["dataset"]["seed_dir"]
dataset_data: str = training_config["dataset"]["data"]
dataset_palmskin_result: str = training_config["dataset"]["palmskin_result"]
# dataset_palmskin_result: str = "28_RGB_m3d"
dataset_base_size: str = training_config["dataset"]["base_size"]
dataset_classif_strategy: str = training_config["dataset"]["classif_strategy"]
dataset_file_name: str = training_config["dataset"]["file_name"]

# dataset_df (finding original image)
dataset_cropped: Path = path_navigator.dbpp.get_one_of_dbpp_roots("dataset_cropped_v3")
src_root = dataset_cropped.joinpath(dataset_seed_dir,
                                    dataset_data,
                                    dataset_palmskin_result,
                                    dataset_base_size)
dataset_file: Path = src_root.joinpath(dataset_classif_strategy,
                                       dataset_file_name)
dataset_df: pd.DataFrame = pd.read_csv(dataset_file, encoding='utf_8_sig')

# show
dataset_df

In [ ]:
tmp_path = Path(list(dataset_df[(dataset_df["image_name"] == testsample)]["path"])[0])
print(testsample)

orig_path = src_root.joinpath(tmp_path)
img_dict["orig"] = ski.io.imread(orig_path)
# tmp_cv_read = cv2.imread(str(orig_path))
# assert np.array_equal(img_dict["orig"], cv2.cvtColor(tmp_cv_read, cv2.COLOR_BGR2RGB))

# show
plot_img_flow(img_dict, ["orig"], dpi=100)
orig_path

In [ ]:
brightness = cv2.cvtColor(img_dict["orig"], cv2.COLOR_RGB2HSV_FULL)[:,:,2]

# create mask (brightness threshold)
intensity = parse_dataset_file_name(dataset_file_name)["intensity"]
print(f"intensity = {intensity}")
img_dict["mask"] = brightness > intensity

# # create mask (color distance threshold)
# zero_img = np.zeros_like(img_dict["orig"], dtype=np.uint16) # 運算時 `orig_img` 會跟著變為 uint16 避免 overflow
# color_distance = np.sqrt(np.sum((img_dict["orig"] - zero_img)**2, axis=2))
# img_dict["mask"] = color_distance > 40

# erode mask
_erode = cv2.erode(np.uint8(img_dict["mask"]*255),
                    img_dict["kernel_ones2x2"], iterations=1)
_dilate = cv2.dilate(_erode, img_dict["kernel_ones2x2"], iterations=1)
img_dict["morph_mask"] = np.bool8(_dilate)

# calculate dark ratio
dark_ratios = []
for bin_img in [img_dict["mask"], img_dict["morph_mask"]]:
    
    pixel_too_dark = np.sum(~bin_img)
    dark_ratio = pixel_too_dark/(bin_img.shape[0]*bin_img.shape[1])
    print(bin_img.shape, "\n", f"dark_ratio = {dark_ratio}")
    dark_ratios.append(dark_ratio)
    
# show
keys = ["orig", "mask", "morph_mask"]
plot_img_flow(img_dict, keys)


""" Functionalize Check """
from modules.dl.cam.analysis import create_brightness_mask

tmp_img, v = create_brightness_mask(img_dict["orig"], intensity)
assert np.array_equal(tmp_img, img_dict["mask"])
assert v == dark_ratios[0]

tmp_img, v = create_brightness_mask(img_dict["orig"], intensity,
                                    erode_kernel=img_dict["kernel_ones2x2"],
                                    erode_iter=1,
                                    dilate_kernel=img_dict["kernel_ones2x2"],
                                    dilate_iter=1)
assert np.array_equal(tmp_img, img_dict["morph_mask"])
assert v == dark_ratios[1]

In [ ]:
cam_name = testsample.replace("crop", "graymap")
cam_path = cam_result_root.joinpath(f"*/grayscale_map/{cam_name}.tiff")


img_dict["cam"] = ski.io.imread(cam_path)
# tmp_cv_read = cv2.imread(str(cam_path), cv2.IMREAD_GRAYSCALE)
# assert np.array_equal(img_dict["cam"], tmp_cv_read)

img_dict["colored_cam"] = \
    cv2.cvtColor(cv2.applyColorMap(img_dict["cam"], cv2.COLORMAP_JET),
                 cv2.COLOR_BGR2RGB)
img_dict["blank"] = np.full_like(img_dict["cam"], 255, dtype=np.uint8)

# show
keys = ["cam", "colored_cam"]
plot_img_flow(img_dict, keys)
cam_path

In [ ]:
_, img_dict["thres(cam)"] = cv2.threshold(img_dict["cam"], cam_threshold, 255, cv2.THRESH_BINARY)
img_dict["thres(cam)*mask"] = img_dict["thres(cam)"]*img_dict["morph_mask"]

# show
keys = ["cam", "thres(cam)", "thres(cam)*mask"]
plot_img_flow(img_dict, keys)


""" Functionalize Check """
from modules.dl.cam.analysis import thres_cam_on_cell_v1

tmp_img = thres_cam_on_cell_v1(img_dict["cam"], cam_threshold, img_dict["morph_mask"])
assert np.array_equal(tmp_img, img_dict["thres(cam)*mask"])

In [ ]:
img_dict["cam*mask"] = img_dict["cam"]*img_dict["morph_mask"]
img_dict["colored_cam*mask"] = img_dict["colored_cam"]*cv2.merge([np.uint8(img_dict["morph_mask"]),]*3)

_, img_dict["thres(cam*mask)"] = cv2.threshold(img_dict["cam*mask"], cam_threshold, 255, cv2.THRESH_BINARY)

# show
keys = ["cam", "cam*mask", "colored_cam*mask", "thres(cam*mask)"]
plot_img_flow(img_dict, keys)


""" Functionalize Check """
from modules.dl.cam.analysis import thres_cam_on_cell_v2

tmp_img = thres_cam_on_cell_v2(img_dict["cam"], cam_threshold, img_dict["morph_mask"])
assert np.array_equal(tmp_img, img_dict["thres(cam*mask)"])

In [ ]:
np.array_equal(img_dict["thres(cam)*mask"], img_dict["thres(cam*mask)"])

In [ ]:
img_dict["erode(thres(cam*mask))"] = cv2.erode(img_dict["thres(cam*mask)"],
                                               img_dict["kernel_ones2x2"],
                                               iterations=1)
img_dict["overlap"] = np.uint8(img_dict["orig"]*0.5 + cv2.merge([img_dict["erode(thres(cam*mask))"]]*3)*0.5)

# show
keys = ["orig", "mask", "morph_mask", "blank", "blank",
        "cam", "blank", "colored_cam", "blank", "blank",
        "blank", "blank", "colored_cam*mask", "blank", "blank",
        "blank", "blank", "thres(cam*mask)", "erode(thres(cam*mask))", "overlap"]
plot_img_flow(img_dict, keys, col=5)


""" Functionalize Check """
from modules.dl.cam.analysis import thres_cam_on_cell_v2

tmp_img = thres_cam_on_cell_v2(img_dict["cam"], cam_threshold,
                               img_dict["morph_mask"],
                               erode_kernel=img_dict["kernel_ones2x2"],
                               erode_iter=1)
assert np.array_equal(tmp_img, img_dict["erode(thres(cam*mask))"])

In [ ]:
print(list(img_dict.keys()))

In [ ]:
# 查找轮廓
contours, _ = cv2.findContours(img_dict["erode(thres(cam*mask))"], cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# 计算每个轮廓的填充面积和像素数
contour_areas = []
contour_pt = []
for contour in contours:
    area = cv2.contourArea(contour)
    if (area > 1):
        contour_areas.append(area)
        contour_pt.append(len(contour))

# 组合成 2 行的数组
result = np.vstack((contour_pt, contour_areas))
print(result, result.shape)


""" Functionalize Check """
from modules.dl.cam.analysis import calc_area

tmp_list = calc_area(img_dict["erode(thres(cam*mask))"])
assert np.array_equal(tmp_list, contour_areas)

In [14]:
""" Functionalize Check """
from modules.dl.cam.analysis import calc_thresed_cam_area_on_cell

tmp_list = calc_thresed_cam_area_on_cell(img_dict["orig"], intensity,
                                            img_dict["cam"], cam_threshold)
assert np.array_equal(tmp_list, contour_areas)